In [17]:
from PIL import Image
import os
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# set results_folder to the folder in the same directory with the name "results"
results_folder = '../results/'

In [9]:
# Loop through every folder in results_folder. For each image inside the folder, add a label to the list of labels.
labels = []
for folder in os.listdir(results_folder):
    for image in os.listdir(results_folder + folder):
        labels.append(folder)
# Create a list of all the images in the results_folder
images = [os.path.join(results_folder, folder, image) for folder in os.listdir(results_folder) for image in os.listdir(results_folder + folder)]

# limit labels and images to 100 items in the list
labels = labels[:100]
images = images[:100]

In [10]:
# Match every image to its label for supervised learning
labels = [labels.index(os.path.basename(os.path.dirname(image))) for image in images]
labels = to_categorical(labels)

In [18]:
# Create a model
model = Sequential()
# Add a convolutional layer with 32 filters, each of size 3x3
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
# dense layer
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# output
model.add(Dense(3, activation='softmax'))

In [19]:
# Compile the model
model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create a generator that will take the data and create batches of images
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(results_folder, target_size=(150, 150), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(results_folder, target_size=(150, 150), batch_size=32, class_mode='categorical')

Found 273728 images belonging to 64 classes.
Found 273728 images belonging to 64 classes.


In [ ]:
model.fit(train_generator, steps_per_epoch=100, epochs=5, validation_data=test_generator, validation_steps=50)